In [1]:
import pandas as pd
import numpy as np 
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
train_df = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


In [3]:
x = train_df[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
y  = train_df["Survived"]

In [4]:
x.isna().sum()

Pclass         0
Sex            0
Age         3292
SibSp          0
Parch          0
Fare         134
Embarked     250
dtype: int64

In [5]:
def data_prep(x,mode):
    x["Age"] =x["Age"].fillna(x["Age"].mean()) 
    x["Fare"] =x["Fare"].fillna(x["Fare"].mean()) 
    x["Embarked"] =x["Embarked"].fillna(x["Embarked"].mode()) 
    le = preprocessing.LabelEncoder()
    x["Sex"] = le.fit_transform(x["Sex"])
    x["Embarked"] = le.fit_transform(x["Embarked"])
    x["Family"] = x["SibSp"] + x["Parch"]
    x["Fare"] =  x['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
    return x
    

In [6]:
x = data_prep(x,"train")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [7]:
x.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,2.106910,0.56114,38.355472,0.397690,0.454560,3.156568,1.504520,0.852250
std,0.837727,0.49625,18.009589,0.862566,0.950076,1.016036,0.835899,1.412792
min,1.000000,0.00000,0.080000,0.000000,0.000000,-0.385662,0.000000,0.000000
25%,1.000000,0.00000,25.000000,0.000000,0.000000,2.306577,1.000000,0.000000
50%,2.000000,1.00000,38.355472,0.000000,0.000000,3.198265,2.000000,0.000000
75%,3.000000,1.00000,53.000000,1.000000,1.000000,3.513335,2.000000,1.000000
max,3.000000,1.00000,87.000000,8.000000,9.000000,6.612928,3.000000,17.000000


In [8]:
x_train,x_val,y_train,y_val = train_test_split(x,y)

In [9]:
model = LogisticRegression()

In [10]:
 model_trained = model.fit(x_train,y_train)

In [11]:
print(classification_report(y_val,model_trained.predict(x_val)))

              precision    recall  f1-score   support

           0       0.80      0.78      0.79     14386
           1       0.71      0.74      0.73     10614

    accuracy                           0.76     25000
   macro avg       0.76      0.76      0.76     25000
weighted avg       0.77      0.76      0.76     25000



In [12]:
print(roc_auc_score(y_val,model_trained.predict(x_val)))

0.7616112195945794


In [13]:
final_model = model.fit(x,y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [14]:
test_df = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")

In [15]:
test_df_clean = test_df[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
test_df_clean = data_prep(test_df_clean,"test")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [16]:
test_df_clean.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Family      0
dtype: int64

In [17]:
preds = final_model.predict(test_df_clean)

In [18]:
preds

array([0, 0, 1, ..., 0, 1, 1])

In [19]:
submit = pd.DataFrame({"PassengerId":test_df["PassengerId"],"Survived":preds})

In [20]:
submit

,PassengerId,Survived
0,100000,0
1,100001,0
2,100002,1
3,100003,0
4,100004,1
...,...,...
99995,199995,1
99996,199996,0
99997,199997,0
99998,199998,1


In [21]:
submit.to_csv("submission.csv",index = False)